In [132]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu
import importlib
import os

In [4]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [11]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)
brandenburg = dataset.load_scene('brandenburg_gate')


In [14]:
# Access data
images_path = brandenburg.images_dir
calibration_data = brandenburg.calibration

In [ ]:
for i in images_path.iterdir():
    print(i)

In [16]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [66]:
# Get two images from images_path
images = list(images_path.iterdir())[:2]
image1, image2 = images[0], images[1]
print(image1, image2)

/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/10624116_2746543073.jpg /Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/96174448_2036661198.jpg


In [68]:
img1 = preprocessor.process_image(image1)
img2 = preprocessor.process_image(image2)

In [51]:
default_sift_config = eu.SIFTConfig()
extractor_config = eu.FeatureExtractorConfig(default_sift_config)
extractor = eu.FeatureExtractor(config=extractor_config)

In [69]:
i1f = extractor.extract_features(img1)
i2f = extractor.extract_features(img2)

In [124]:
bf_config = esu.BFMatcherConfig(knn=True)
matcher_config = esu.MatcherConfig(bf_config)

In [125]:
matcher = esu.FeatureMatcher(matcher_config)

In [126]:
matches = matcher.match_features(i1f, i2f)

In [127]:
valid, kp1, kp2 = matcher.filter_lowe_matches(matches, i1f, i2f)

In [141]:
type(kp1[0])



cv2.KeyPoint

In [142]:
importlib.reload(esu)

<module 'utils.estimator_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/estimator_util.py'>

In [143]:
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)


In [144]:
fe = esu.FundamentalMatrixEstimator(kp1, kp2, es_conf)

In [145]:
fe.estimate()

array([[ 2.19455369e-06, -1.28450711e-04,  5.75317978e-02],
       [ 1.10165394e-04,  6.54828648e-07, -2.44261228e-02],
       [-6.91245930e-02,  3.13106405e-02,  1.00000000e+00]])

In [149]:
inl = fe.get_inliers()

In [ ]:
# Convert inliers to a set of tuples for easy comparison
inliers_set = set(map(tuple, inl[0]))
print(f"Found {len(inliers_set)} inliers from {len(inl[0])} inliers")

# Find outliers by checking which keypoints in kp1 are not in inliers
outliers = [kp for kp in kp1 if (kp.pt[0], kp.pt[1]) not in inliers_set]

print(f"Inliers count: {len(inliers_set)} out of {len(kp1)}")
print("Total outliers found: ")
print(len(set(outliers)))

Found 24 inliers from 28 INLIERS
Inliers count: 24 out of 59
Total outliers found: 
31
